In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [53]:
training = pd.read_csv('processedTrain.csv')
testing = pd.read_csv('processedTest.csv')

In [54]:
X_train = training.drop(['prognosis'],axis=1) # the training attributes
y_train = training['prognosis'] # the training labels

X_test = testing.drop(['prognosis'],axis=1) # the testing attributes
y_test = testing['prognosis'] # the testing labels

In [55]:
y_train_enc= pd.get_dummies(y_train)
y_test_enc= pd.get_dummies(y_test)

In [56]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [57]:
model= Sequential()
model.add(Dense(64, activation='relu', input_shape= (X_train.shape[1], )))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_train_enc.shape[1], activation='softmax'))

model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
early_stopping_monitor= EarlyStopping(patience=10, monitor='val_accuracy')
model.fit(X_train, y_train_enc, batch_size=120, epochs=30, validation_data=(X_test,y_test_enc), callbacks=[early_stopping_monitor])

Epoch 1/30
41/41 [==============================] - 0s 3ms/step - loss: 3.5137 - accuracy: 0.1159 - val_loss: 3.1758 - val_accuracy: 0.4048
Epoch 2/30
41/41 [==============================] - 0s 1ms/step - loss: 2.6135 - accuracy: 0.5081 - val_loss: 1.8507 - val_accuracy: 0.6429
Epoch 3/30
41/41 [==============================] - 0s 1ms/step - loss: 1.2158 - accuracy: 0.8020 - val_loss: 0.5913 - val_accuracy: 0.9762
Epoch 4/30
41/41 [==============================] - 0s 1ms/step - loss: 0.3004 - accuracy: 0.9982 - val_loss: 0.1805 - val_accuracy: 0.9762
Epoch 5/30
41/41 [==============================] - 0s 1ms/step - loss: 0.0702 - accuracy: 1.0000 - val_loss: 0.1339 - val_accuracy: 0.9762
Epoch 6/30
41/41 [==============================] - 0s 1ms/step - loss: 0.0305 - accuracy: 1.0000 - val_loss: 0.1306 - val_accuracy: 0.9762
Epoch 7/30
41/41 [==============================] - 0s 1ms/step - loss: 0.0180 - accuracy: 1.0000 - val_loss: 0.1307 - val_accuracy: 0.9762
Epoch 8/30
41/41 [==

In [59]:
model.evaluate(X_test, y_test_enc, batch_size=1, steps=5)

5/5 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 1.0000


[0.0011228186776861548, 1.0]